###MLOps: Assembling Pipelines

In this project, our primary focus will be to create COLUMN TRANSFORMERS to clean data, and then create an ML PIPELINE to ensure whatever new data is fed into the model, goes through the pipeline and hence is fed into the model with required TRANSFORMATIONS!

In [0]:
# notebook config
USER_NAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
FILE_STORE_ROOT = '/FileStore/shared_uploads/'+USER_NAME

In [0]:
spend = spark.read.csv(
  FILE_STORE_ROOT+'/wholesale_customers/', 
  header=True, 
  inferSchema=True
  )

spend_df = spend.toPandas()

WE NEED TO ENSURE THAT IN OUR PIPELINES WE ARE REFERENCING OUR COLUMNS BY INDICES RATHER THAN COLUMN NAMES, THIS IS VERY IMPORTANT CAUSE...
1. After transformation through column transformers in the pipeline, the columns will be renamed to indices
2. Also, as move towards model deployment, it's important to refer columns with indices rather than names

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# defining stages for ColumnTransformer
transformer = ColumnTransformer([
  ('ohe_encoder', OneHotEncoder( drop='first', sparse=False), [0, 1]), # apply OHE to channel and region fields
  ('robust_scaler', RobustScaler(), [2,3,4,5,6,7]) # apply robust scaler to all other fields
  ])

## applying transformations
#X = transformer.fit_transform( spend_df )

In [0]:
from sklearn.cluster import KMeans

# instantiating and configure clustering model
km = KMeans(
  n_clusters=4, 
  init='random',
  n_init=1000
  )


Ideally, hyperparameter tuning should be done! Please refer to the other notebook for the same!

In [0]:
from sklearn.pipeline import Pipeline

# combining transformations with model as a pipeline
clf = Pipeline(
  steps=[
    ('transform', transformer),
    ('clustering', km)
    ]
  )

Important Points...
1. Inside a pipeline, steps will be executed in the order specified. Unlike a column transformer. Hence it's important to specify the steps correctly
2. Indices for transformations have to be according to the initial dataset. Although after a column transformation column indices will be reset, the pipeline will refer to the original dataset for recognizing the columns

In [0]:
from sklearn.metrics import silhouette_score

# fitting the model
clf.fit(spend_df)

# generating a prediction
predict = clf.predict(spend_df)

# scoring the predictions
print( silhouette_score( spend_df, predict) )

0.42248161797300304


Although the accuracy is not great, we were able to set a pipeline.

Please refer to my code below for a more complex pipeline -->

In [0]:
# after making data available at /dbfs/tmp/melbourne/melb_data.csv
df = spark.read.csv(
  FILE_STORE_ROOT+'/melbourne_housing/melb_data.csv', 
  sep=',', 
  header=True,
  inferSchema=True
  ).toPandas()

In [0]:
import pandas as pd
import numpy as np


# separating features from label column
features = df.drop('Price', axis=1)
labels = df['Price']

With the Melbourne data loaded into two DataFrames, one representing our features and the other representing our labels, let's assemble our pipeline including the two steps for feature transformation. Notice again that we are not using field names but instead using positional references for our fields:

In [0]:
features

,handicap,water,adoption,medical,elsalvador,religion,satellite,nicraguan,missle,immigration,fuel,education,superfund,crime,exports,southafrica
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
431,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
432,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [0]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# defining stages for missing value ColumnTransformer
missing_value_transformer = ColumnTransformer([
  (  'median_missing', 
      SimpleImputer(missing_values=np.NaN, strategy='median'), 
      [3,10,11,12,13,14,15]
  ),
  (  'most_frequent_missing', 
      SimpleImputer(missing_values=np.NaN, strategy='most_frequent'), 
      [4, 5]
  )
  ])

# defining stages for encoding & scaling ColumnTransformer
encoding_scaling_transformer = ColumnTransformer([
  ('get my previously transformed stuff', 'passthrough', list(range(0,4))+[6]),
  ('ohe_encode', OneHotEncoder( drop='first', sparse=False), [7, 8]),
  ('normalize', RobustScaler(), [4, 5])
  ])

# instantiating and configure model
reg = LinearRegression()

# defining pipeline
clf = Pipeline(steps=[
  ('missing_values', missing_value_transformer),
  ('encoding_scaling', encoding_scaling_transformer),
  ('regression', reg)
  ])

We now have a pipeline which applies two steps of data transformation before passing data to a linear regression model.  Let's now execute the pipeline to train and score our model:

In [0]:
# fit the model
clf.fit(features, labels)

# make predictions
predicted_prices = clf.predict(features)

# calculate score
print( clf.score(features, labels) )

0.35844407788242505


While not a great score, our Pipeline object allows us to manage a series of transformations in combination with our model, easing our logic management. 

The most important reson for a pipeline in my opinion is to ensure whatever new data is fed into the model for predictions in the future goes through the required transformations to make it similar to the training data used!!!